# 추천 시스템 - 아이템 기반 협업 필터링(item based collaborative filtering)

이번 글은 추천 시스템에서 아이템 기반 협업 필터링에 관해 작성합니다.  
해당 자료는 아래에서 참고했습니다.

- https://www.kaggle.com/manisha1409raj/recommender-system-collaborative-filtering
- https://medium.com/sfu-big-data/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0
- https://www.coursera.org/learn/collaborative-filtering/
- https://www.kaggle.com/rounakbanik/movie-recommender-systems
- https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system


데이터는 kaggle의 **The movies Dataset (https://www.kaggle.com/rounakbanik/the-movies-dataset)** 을 사용했습니다.

# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
data = pd.read_csv('./movie_data/ratings_small.csv')

In [12]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


지금은 userId 컬럼과 movieId 컬럼이 따로따로 존재합니다.   
아이템 기반 협업 필터링(item based collaborative filtering) 기반으로 추천 시스템을 만드려면 user-item 테이블로 만들어주어야 합니다.

pivot_table을 이용해서 만들어주겠습니다.

In [14]:
data = data.pivot_table('rating', index = 'userId', columns = 'movieId')

In [15]:
data.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.shape

(671, 9066)

그러면 위와 같이 사용자 아이디 별 영화에 대한 평점을 볼 수 있는 테이블이 만들어지게 됩니다.

근데 문제는 여기서 영화 아이디는 있지만 영화 title이 없다는 문제가 있습니다. 영화 title을 가져와서 merge하겠습니다.

In [25]:
ratings = pd.read_csv('./movie_data/ratings_small.csv')
movies = pd.read_csv('./movie_data/tmdb_5000_movies.csv')


In [28]:
movies.rename(columns = {'id': 'movieId'}, inplace = True)

In [29]:
ratings_movies = pd.merge(ratings, movies, on = 'movieId')

In [30]:
ratings_movies.head(1)

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296


In [31]:
ratings_movies.shape

(18571, 23)

In [35]:
data = ratings_movies.pivot_table('rating', index = 'userId', columns = 'title').fillna(0)

In [36]:
data.head()

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
data.shape

(670, 856)

그러면 위와 같이 사용자별 영화 title에 따라 평점을 매긴 정보를 볼 수 있게 됩니다.

근데 문제가 있습니다. 아이템 기반 협업 필터링이므로 row가 user 기반이면 안됩니다.  
item이 row가 되어야 합니다. 왜냐하면 cosine similarity를 구할 때 row 기반으로 유사도를 측정하기 때문이죠.  

그래서 row를 item으로 변경합니다.

In [38]:
data = data.transpose()
data.head(2)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
data.shape

(856, 670)

이렇게 변경이 되었습니다.

이제 item별로 유사한 것을 측정합니다.

In [40]:
movie_sim = cosine_similarity(data, data)
print(movie_sim.shape)

(856, 856)


In [42]:
movie_sim_df = pd.DataFrame(data = movie_sim, index = data.index, columns = data.index)

In [43]:
movie_sim_df.head(3)

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.0,0.0,0.0,0.182153,0.0,0.022069,0.085323,0.0,0.0,0.10349,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.0,0.0,0.014121
12 Angry Men,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1408,0.0,0.0,1.0,0.447214,0.0,0.173381,0.028245,0.0,0.0,0.00000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.0,0.0,0.000000


그러면 이제 특정 영화와 비교했을 때 그 영화와 유사한 영화들을 추천해주면 됩니다.

In [47]:
movie_sim_df["X-Men Origins: Wolverine"].sort_values(ascending=False)[1:10]

title
Romeo Must Die                        0.649625
The Wedding Planner                   0.631669
Dogtown and Z-Boys                    0.501189
An Unfinished Life                    0.485643
Conquest of the Planet of the Apes    0.474626
Reign Over Me                         0.458155
The Terminal                          0.445337
Young Frankenstein                    0.423840
Whale Rider                           0.394136
Name: X-Men Origins: Wolverine, dtype: float64

In [58]:
movie_sim_df["Harry Potter and the Half-Blood Prince"].sort_values(ascending=False)[1:10]

title
Liar Liar                                 1.000000
Family Plot                               1.000000
Once                                      1.000000
Synecdoche, New York                      1.000000
Rendition                                 1.000000
Harry Potter and the Half-Blood Prince    1.000000
The Astronaut Farmer                      0.970143
Schindler's List                          0.724286
The Last King of Scotland                 0.707107
Name: Harry Potter and the Half-Blood Prince, dtype: float64

In [56]:
movie_sim_df["Harry Potter and the Half-Blood Prince"].sort_values(ascending=False)[:10]

title
The Blue Lagoon                           1.000000
Liar Liar                                 1.000000
Family Plot                               1.000000
Once                                      1.000000
Synecdoche, New York                      1.000000
Rendition                                 1.000000
Harry Potter and the Half-Blood Prince    1.000000
The Astronaut Farmer                      0.970143
Schindler's List                          0.724286
The Last King of Scotland                 0.707107
Name: Harry Potter and the Half-Blood Prince, dtype: float64

In [61]:
movie_sim_df["King Kong"].sort_values(ascending=False)[1:10]

title
Fantasia                                  0.648886
2046                                      0.648886
Rendition                                 0.486664
Family Plot                               0.486664
Liar Liar                                 0.486664
Harry Potter and the Half-Blood Prince    0.486664
Synecdoche, New York                      0.486664
The Blue Lagoon                           0.486664
Once                                      0.486664
Name: King Kong, dtype: float64

개인적으로 봤을 때 성능이 썩 좋지는 않습니다.

그 이유는 2개의 데이터 셋이 조금 다르기 때문입니다. ratings와 movies 데이터가 달라서(ratings가 small이라 데이터가 적습니다. 그렇다고 ratings.csv를 사용하자니 데이터가 큽니다.) 모든 영화 정보를 포함하지 않기 때문이죠.

이를 보완하기 위해서 Movielens 데이터를 이용해서 사용해보려고 합니다.

# Movielen 데이터 사용

**https://www.kaggle.com/sengzhaotoo/movielens-small** 이 데이터입니다.

여기서의 커널은 아래 리스트를 참고했습니다.  
- https://www.kaggle.com/manisha1409raj/recommender-system-collaborative-filtering 
- https://www.kaggle.com/rajmehra03/cf-based-recsys-by-low-rank-matrix-factorization


In [15]:
rating_data = pd.read_csv('./movie_lens/ratings.csv')
movie_data = pd.read_csv('./movie_lens/movies.csv')

In [16]:
rating_data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [17]:
movie_data.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


이 두 개의 파일은 현재 사용자-평점 데이터와 영화 데이터가 따로 나뉘어져 있습니다.

그러나 이 두 개의파일은 movieId를 공통으로 가지고 있으니 movieId를 기준으로 합쳐줄 수 있습니다.  
사용자-영화에 따른 평점 데이터를 pivot_table로 만들겠습니다.

In [18]:
rating_data.drop('timestamp', axis = 1, inplace=True)
rating_data.head(2)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0


In [19]:
user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')

In [20]:
user_movie_rating.head(2)

,userId,movieId,rating,title,genres
0,1,31,2.5,Dangerous Minds (1995),Drama
1,7,31,3.0,Dangerous Minds (1995),Drama


In [21]:
movie_user_rating = user_movie_rating.pivot_table('rating', index = 'title', columns='userId')
user_movie_rating = user_movie_rating.pivot_table('rating', index = 'userId', columns='title')

In [22]:
user_movie_rating.head(5)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
movie_user_rating.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$9.99 (2008),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Neath the Arizona Skies (1934),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


하나는 컬럼이 영화, 다른 하나는 컬럼이 사용자입니다.  

자! 이 상태에서 먼저 아이템 기반 협업 필터링 (item based collaborative filtering)을 진행해보겠습니다.


일단, NaN 값은 0으로 바꿔주죠

In [24]:
movie_user_rating.fillna(0, inplace = True)
movie_user_rating.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


현재, row가 영화, col이 user로 되어있습니다.  

아이템 기반 협업 필터링은 ~ 상품을 구매한 고객들은 다음 상품도 구매했다. 라는 뜻입니다.  
즉, ~ 영화를 본 고객들은 이런 영화도 봤다. 라고 해석할 수 있겠네요.  

그리고 그 기반은 평점이 비슷한 것을 기반으로 합니다.  
이제 그 평점이 비슷하다는 것을 코사인 유사도로 측정해서 확인해봅시다.

In [25]:
item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.05821787, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.05821787, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [26]:
print(movie_user_rating.shape)
print(item_based_collabor.shape)

(9064, 671)
(9064, 9064)


In [28]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)

In [29]:
item_based_collabor.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
$9.99 (2008),0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,...,0.0,0.0,0.0,0.000000,0.0,0.013899,0.0,0.058218,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
'Neath the Arizona Skies (1934),0.164399,0.0,0.0,1.000000,0.124035,0.0,0.085436,0.000000,0.0,0.019259,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
'Round Midnight (1986),0.020391,0.0,0.0,0.124035,1.000000,0.0,0.010597,0.143786,0.0,0.136163,...,0.0,0.0,0.0,0.121567,0.0,0.000000,0.0,0.000000,0.0,0.0


이렇게 Item 간의 유사도가 측정이 되었습니다!

이제 특정 item에 있어 비슷한 item을 추천해주는 기능을 넣으면 됩니다.

In [32]:
def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]

In [33]:
get_item_based_collabor('Godfather, The (1972)')

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.773685
Goodfellas (1990)                            0.620349
One Flew Over the Cuckoo's Nest (1975)       0.568244
American Beauty (1999)                       0.557997
Star Wars: Episode IV - A New Hope (1977)    0.546750
Name: Godfather, The (1972), dtype: float64

이렇게 결과가 나왔음을 확인할 수 있습니다.